In [104]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
from sklearn.ensemble import RandomForestRegressor
from datetime import date


## Spotify API Call

In [79]:
client_id = '83c1f666855f45f89b2216e55c7bf535'
client_secret = '9bfd86b3fd3c43ada84e1f282fb1df35'
redirect_uri='http://localhost:8910/callback'
username = '61d4pmnwnwjt7xz9tj6v3txho'

scope = 'user-top-read'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, 
                                                      client_secret=client_secret)
                                                      
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

token = util.prompt_for_user_token(username,scope, client_id, client_secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)
    

## Top Tracks Extraction

In [5]:
results = sp.current_user_top_tracks(limit=50, offset=0,time_range='short_term')

#Convert the results into a dataframe
track_name = []
track_id = []
artist = []
album = []
duration = []
popularity = []
for i, items in enumerate(results['items']):
        track_name.append(items['name'])
        track_id.append(items['id'])
        artist.append(items["artists"][0]["name"])
        duration.append(items["duration_ms"])
        album.append(items["album"]["name"])
        popularity.append(items["popularity"])

# Create the final df   
df_top_tracks = pd.DataFrame({ "track_name": track_name, 
                             "album": album, 
                             "track_id": track_id,
                             "artist": artist, 
                             "duration": duration, 
                             "popularity": popularity})

df_top_tracks

,track_name,album,track_id,artist,duration,popularity
0,Vaathi Coming,Master (Original Motion Picture Soundtrack),2BcPFQ7nrtUObgAs72xaac,Anirudh Ravichander,228257,63
1,"Ey Inge Paaru (From ""Velaiyilla Pattathari"")",Voice of Ani,34hssXUwRCru1RbfxCWLth,Anirudh Ravichander,117788,24
2,Feels Like Love,Noah Schnacky EP,04hHPq6kXTbcSSDrdaP3s4,Noah Schnacky,174760,53
3,Andha Kanna Paathaakaa,Master (Original Motion Picture Soundtrack),0qvoxfYodIfe14gaidhnsV,Anirudh Ravichander,194500,65
4,All the Cowboys,All the Cowboys,13TOvHAfdO8wryqxBL7fnI,Alexandra Kay,228231,55
5,My Person,Wilderness,1MOOJuxUu9QiQE9GgkYYPb,Spencer Crandall,177000,59
6,Comeback,Noah Schnacky EP,1wr0HUe5tFDlN32jfwt9IS,Noah Schnacky,168240,49
7,Maybe We Will - 2020 Version,Noah Schnacky EP,26LKaGfw6ZolgGArPDEN2R,Noah Schnacky,190440,43
8,Polakattum Para Para,Master (Original Motion Picture Soundtrack),2K058s9yrpoUfANaQt7Zu4,Anirudh Ravichander,214124,55
9,"Neeyum Naanum (From ""Naanum Rowdy Dhaan"")",Voice of Ani,2XMJ3A06ex0UI9JRPeOYrQ,Anirudh Ravichander,301951,21


## Features Extraction

In [6]:
def get_features(sp,df):
    playlist = df[['track_id','track_name']]
    features = []
    
    features += sp.audio_features(playlist.iloc[0:50, 0])

    feature_list = []
    for feature in features:
        feature_list.append([feature['danceability'],
                        feature['energy'],
                        feature['key'],
                        feature['loudness'],
                        feature['mode'],
                        feature['speechiness'],
                        feature['acousticness'],
                        feature['instrumentalness'],
                        feature['liveness'],
                        feature['valence'],
                        feature['tempo'],
                        feature['duration_ms']]
                       )

    df_audio_feature = pd.DataFrame(feature_list, columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 
                                                          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                          'duration_ms'])
    df_playlist_audio_features = pd.concat([playlist, df_audio_feature], axis=1)
    return df_playlist_audio_features
    

In [7]:
df_final_features = get_features(sp,df_top_tracks)
#Drop the mode feature as it is binary
del df_final_features['mode']
df_final_features

,track_id,track_name,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
0,2BcPFQ7nrtUObgAs72xaac,Vaathi Coming,0.643,0.952,7,-4.150,0.2460,0.27600,0.490000,0.0588,0.585,164.784,228258
1,34hssXUwRCru1RbfxCWLth,"Ey Inge Paaru (From ""Velaiyilla Pattathari"")",0.643,0.834,7,-4.104,0.1820,0.06520,0.000042,0.6120,0.872,189.998,117788
2,04hHPq6kXTbcSSDrdaP3s4,Feels Like Love,0.639,0.845,2,-5.139,0.0414,0.00117,0.000000,0.1260,0.703,117.047,174760
3,0qvoxfYodIfe14gaidhnsV,Andha Kanna Paathaakaa,0.747,0.860,6,-4.071,0.2790,0.30100,0.038300,0.0743,0.625,90.445,194500
4,13TOvHAfdO8wryqxBL7fnI,All the Cowboys,0.501,0.554,9,-6.292,0.0265,0.32100,0.000000,0.1780,0.470,144.022,228232
5,1MOOJuxUu9QiQE9GgkYYPb,My Person,0.567,0.753,7,-5.291,0.0487,0.08940,0.000000,0.1300,0.756,165.966,177000
6,1wr0HUe5tFDlN32jfwt9IS,Comeback,0.542,0.817,11,-5.192,0.0324,0.03830,0.000000,0.2180,0.683,95.978,168240
7,26LKaGfw6ZolgGArPDEN2R,Maybe We Will - 2020 Version,0.685,0.485,4,-7.755,0.0272,0.15100,0.000000,0.1100,0.526,85.045,190440
8,2K058s9yrpoUfANaQt7Zu4,Polakattum Para Para,0.758,0.796,11,-6.483,0.1700,0.16500,0.000041,0.2900,0.919,145.475,214125
9,2XMJ3A06ex0UI9JRPeOYrQ,"Neeyum Naanum (From ""Naanum Rowdy Dhaan"")",0.572,0.620,9,-9.949,0.0436,0.10100,0.004880,0.1130,0.345,157.968,301951


## EDA Performed

## Featured Playlists Extraction

In [8]:
def featured_playlists(sp):
    id = []
    name = []
    num_tracks = []
    
    featured = sp.featured_playlists()
    playlists = featured['playlists']
    for i, items in enumerate(playlists['items']):
        id.append(items['id'])
        name.append(items['name'])
        num_tracks.append(items['tracks']['total'])
    
    df_playlists = pd.DataFrame({'playlist_id':id, 'playlist_name':name, '#tracks': num_tracks})
    return df_playlists

In [9]:
df_featured_playlists = featured_playlists(sp)
df_featured_playlists

,playlist_id,playlist_name,#tracks
0,37i9dQZF1DX1s9knjP51Oa,Calm Vibes,236
1,37i9dQZF1DWZd79rJ6a7lp,Sleep,257
2,37i9dQZF1DX4bSrsRWE9cd,Bliss,74
3,37i9dQZF1DWTwnEm1IYyoj,Soft Pop Hits,232
4,37i9dQZF1DX8ymr6UES7vc,Rain Sounds,166
5,37i9dQZF1DXbcPC6Vvqudd,Night Rain,217
6,37i9dQZF1DX4wta20PHgwo,Late Night Jazz,80
7,37i9dQZF1DWUAeTOoyNaqm,ASMR Sleep,64
8,37i9dQZF1DWU0ScTcjJBdj,Relax & Unwind,100
9,37i9dQZF1DXcCnTAt8CfNe,Musical Therapy,176


## Fetch Tracks for each playlists

In [10]:
def get_playlist_tracks(sp, playlist_id):
    tracks = []
    offset = 0
    while True:
        track_list = sp.playlist_tracks(playlist_id, fields = None, limit = 100, offset = offset, market = None)
        tracks += track_list['items']
        
        if track_list['next'] is not None:
            offset +=100
        else:
            break
            
    track_id = []
    track_name = []
    
    for track in tracks:
        track_id.append(track['track']['id'])
        track_name.append(track['track']['name'])
    
    df_playlist_tracks = pd.DataFrame({'track_id':track_id, 'track_name': track_name})
    return df_playlist_tracks
    

## Get Audio Features for each track within a playlist

In [11]:
def get_audio_features(sp, playlist_id):
    playlist = get_playlist_tracks(sp, playlist_id)
    audio_features = []
    index = 0
    while index < playlist.shape[0]:
        audio_features += sp.audio_features(playlist.iloc[index:index + 50, 0])
        index += 50
    
    feature_list = []
    for feature in audio_features:
        feature_list.append([feature['danceability'],
                        feature['energy'],
                        feature['key'],
                        feature['loudness'],
#                         feature['mode'],
                        feature['speechiness'],
                        feature['acousticness'],
                        feature['instrumentalness'],
                        feature['liveness'],
                        feature['valence'],
                        feature['tempo'],
                        feature['duration_ms']]
                       )

    df_audio_feature = pd.DataFrame(feature_list, columns = ['danceability', 'energy', 'key', 'loudness', 'speechiness', 
                                                          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
                                                          'duration_ms'])
    df_playlist_audio_features = pd.concat([playlist, df_audio_feature], axis=1)
    return df_playlist_audio_features
    

## Get the mean & merge each playlist

In [12]:
def get_audio_features_mean (sp, playlist_id):
    playlist = get_audio_features(sp, playlist_id)
    df_mean_playlist = pd.DataFrame(playlist.mean(),columns = [playlist_id])
    return df_mean_playlist

In [13]:
dataframes = []
for i in df_featured_playlists['playlist_id']:
    dataframes.append(get_audio_features_mean(sp, i))
    
dataframes
    

[                  37i9dQZF1DX1s9knjP51Oa
 danceability                    0.420339
 energy                          0.030545
 key                             4.466102
 loudness                      -27.924229
 speechiness                     0.051596
 acousticness                    0.992758
 instrumentalness                0.933657
 liveness                        0.105367
 valence                         0.264685
 tempo                         100.236153
 duration_ms                158461.186441,
                   37i9dQZF1DWZd79rJ6a7lp
 danceability                    0.185260
 energy                          0.038644
 key                             4.863813
 loudness                      -30.057572
 speechiness                     0.041844
 acousticness                    0.951167
 instrumentalness                0.886101
 liveness                        0.106140
 valence                         0.083682
 tempo                          84.251650
 duration_ms                18530

In [14]:
#Combines the 
X = reduce(lambda left,right: pd.merge(left,right, left_index=True, right_index=True), dataframes)
X

,37i9dQZF1DX1s9knjP51Oa,37i9dQZF1DWZd79rJ6a7lp,37i9dQZF1DX4bSrsRWE9cd,37i9dQZF1DWTwnEm1IYyoj,37i9dQZF1DX8ymr6UES7vc,37i9dQZF1DXbcPC6Vvqudd,37i9dQZF1DX4wta20PHgwo,37i9dQZF1DWUAeTOoyNaqm,37i9dQZF1DWU0ScTcjJBdj,37i9dQZF1DXcCnTAt8CfNe,37i9dQZF1DXcxvFzl58uP7,37i9dQZF1DX0jgyAiPl8Af
danceability,0.420339,0.185260,0.495473,0.577621,0.134352,0.143710,0.442063,0.476612,0.521310,0.286309,0.660590,0.594400
energy,0.030545,0.038644,0.235396,0.571837,0.737903,0.735210,0.173319,0.270312,0.363995,0.046274,0.489220,0.118244
key,4.466102,4.863813,3.770270,5.366379,4.144578,4.055300,4.850000,3.687500,5.810000,4.414773,4.700000,4.962500
loudness,-27.924229,-30.057572,-14.778986,-6.833733,-19.989120,-21.869972,-18.941112,-34.300563,-11.082280,-26.769313,-9.246250,-17.684996
speechiness,0.051596,0.041844,0.038682,0.050501,0.074907,0.083270,0.041355,0.224439,0.038126,0.040317,0.080760,0.071524
acousticness,0.992758,0.951167,0.816699,0.335439,0.281567,0.231259,0.872147,0.271781,0.695853,0.957290,0.393581,0.961775
instrumentalness,0.933657,0.886101,0.164258,0.004962,0.854964,0.869419,0.708216,0.590622,0.074661,0.904602,0.143667,0.871458
liveness,0.105367,0.106140,0.133188,0.139762,0.665024,0.596056,0.135934,0.223853,0.155497,0.107427,0.176014,0.109330
valence,0.264685,0.083682,0.248828,0.422695,0.008342,0.016355,0.220092,0.082288,0.297808,0.130684,0.528421,0.239324
tempo,100.236153,84.251650,117.758108,117.667388,89.427892,90.601189,106.381312,100.474734,117.174500,93.196159,116.973320,109.223550


In [15]:
y = pd.DataFrame(df_final_features.mean(), columns = ['top_playlist'])
y

,top_playlist
danceability,0.671060
energy,0.754080
key,5.200000
loudness,-5.817680
speechiness,0.106166
acousticness,0.235979
instrumentalness,0.033735
liveness,0.207758
valence,0.661100
tempo,120.519860


## Memory Based Collaborative Filter

### Cosine Similarity

In [23]:
def cos_sim(a,b):
    dot_product = np.dot(a,b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product/(norm_a * norm_b)

In [24]:
#Cosine similarity does not look at the magnititudes of individual elements within the vector, just the angle
def similarities(user_playlist,featured_playlist):
    x = cos_sim(user_playlist,featured_playlist)
    return x

In [25]:
def cos_dataframe(user_playlist):
    featured = []
    distance_score = []
    for i in df_featured_playlists['playlist_id']:
        featured.append(i)
        distance_score.append(similarities(user_playlist,X[i]))
    df_cos_dist = pd.DataFrame({'featured_playlist':featured, 'distance_score':distance_score})
    sort_values = df_cos_dist.sort_values('distance_score', ascending = True)
    return sort_values      

In [26]:
cos_dataframe(y['top_playlist'])

,featured_playlist,distance_score
6,37i9dQZF1DX4wta20PHgwo,1.0
0,37i9dQZF1DX1s9knjP51Oa,1.0
1,37i9dQZF1DWZd79rJ6a7lp,1.0
7,37i9dQZF1DWUAeTOoyNaqm,1.0
11,37i9dQZF1DX0jgyAiPl8Af,1.0
9,37i9dQZF1DXcCnTAt8CfNe,1.0
5,37i9dQZF1DXbcPC6Vvqudd,1.0
4,37i9dQZF1DX8ymr6UES7vc,1.0
10,37i9dQZF1DXcxvFzl58uP7,1.0
8,37i9dQZF1DWU0ScTcjJBdj,1.0


### Euclidean Distance

In [27]:
#Euclidian distance takes the magnitude of each element within the vector into consideration 
def euclidean_distance(user_playlist,featured_playlist):    
    x = np.sqrt(np.sum((user_playlist - featured_playlist) ** 2))
    return x

In [28]:
def euc_dataframe(user_playlist):
    featured = []
    distance_score = []
    for i in df_featured_playlists['playlist_id']:
        featured.append(i)
        distance_score.append(euclidean_distance(user_playlist,X[i]))
    df_euc_dist = pd.DataFrame({'featured_playlist':featured, 'distance_score':distance_score})
    sort_values = df_euc_dist.sort_values('distance_score', ascending = True)
    return sort_values

In [29]:
euc_dataframe(y['top_playlist'])

,featured_playlist,distance_score
2,37i9dQZF1DX4bSrsRWE9cd,8869.737011
3,37i9dQZF1DWTwnEm1IYyoj,11645.267124
7,37i9dQZF1DWUAeTOoyNaqm,14386.999146
8,37i9dQZF1DWU0ScTcjJBdj,18784.211060
10,37i9dQZF1DXcxvFzl58uP7,34008.460363
4,37i9dQZF1DX8ymr6UES7vc,35899.387142
5,37i9dQZF1DXbcPC6Vvqudd,36066.300445
1,37i9dQZF1DWZd79rJ6a7lp,37069.146790
9,37i9dQZF1DXcCnTAt8CfNe,45890.116130
11,37i9dQZF1DX0jgyAiPl8Af,56549.291557


In [49]:
forest = RandomForestRegressor(max_depth=5, max_features=10)
forest.fit(X,y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]

for i in range(len(importances)):
    print("%d. %s %f" %(i + 1, X.columns[i], importances[indices[i]]))

1. 37i9dQZF1DX1s9knjP51Oa 0.120044
2. 37i9dQZF1DWZd79rJ6a7lp 0.099954
3. 37i9dQZF1DX4bSrsRWE9cd 0.099866
4. 37i9dQZF1DWTwnEm1IYyoj 0.096681
5. 37i9dQZF1DX8ymr6UES7vc 0.093856
6. 37i9dQZF1DXbcPC6Vvqudd 0.090188
7. 37i9dQZF1DX4wta20PHgwo 0.086440
8. 37i9dQZF1DWUAeTOoyNaqm 0.080053
9. 37i9dQZF1DWU0ScTcjJBdj 0.079487
10. 37i9dQZF1DXcCnTAt8CfNe 0.060186
11. 37i9dQZF1DXcxvFzl58uP7 0.053271
12. 37i9dQZF1DX0jgyAiPl8Af 0.039973


<ipython-input-49-118c6a44abf4>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  forest.fit(X,y)


In [52]:
for i, playlist in enumerate(df_featured_playlists['playlist_id']):
    try:
        string_command = "df_{} = get_audio_features(sp, playlist)".format(playlist)
        print("Create {}".format(string_command))
        exec(string_command)
    except:
        print("playlist with id {} is not valid, skiping ".format(playlist))
        pass

Create df_37i9dQZF1DX1s9knjP51Oa = get_audio_features(sp, playlist)
Create df_37i9dQZF1DWZd79rJ6a7lp = get_audio_features(sp, playlist)
Create df_37i9dQZF1DX4bSrsRWE9cd = get_audio_features(sp, playlist)
Create df_37i9dQZF1DWTwnEm1IYyoj = get_audio_features(sp, playlist)
Create df_37i9dQZF1DX8ymr6UES7vc = get_audio_features(sp, playlist)
Create df_37i9dQZF1DXbcPC6Vvqudd = get_audio_features(sp, playlist)
Create df_37i9dQZF1DX4wta20PHgwo = get_audio_features(sp, playlist)
Create df_37i9dQZF1DWUAeTOoyNaqm = get_audio_features(sp, playlist)
Create df_37i9dQZF1DWU0ScTcjJBdj = get_audio_features(sp, playlist)
Create df_37i9dQZF1DXcCnTAt8CfNe = get_audio_features(sp, playlist)
Create df_37i9dQZF1DXcxvFzl58uP7 = get_audio_features(sp, playlist)
Create df_37i9dQZF1DX0jgyAiPl8Af = get_audio_features(sp, playlist)


## Create Dataframe with top 3 playlists

In [84]:
frames = [df_37i9dQZF1DX4bSrsRWE9cd, df_37i9dQZF1DWTwnEm1IYyoj, df_37i9dQZF1DWUAeTOoyNaqm]
df =pd.concat(frames)
df = df.set_index(['track_id'])
del df['track_name']
df

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms
track_id,,,,,,,,,,,
7qH9Z4dJEN0l9bidizW7fq,0.457,0.4070,9,-11.475,0.0308,0.8050,0.884000,0.1260,0.12600,117.053,257533
28sWYevC75ZrUMJ0tD4zWM,0.521,0.2650,1,-18.229,0.0319,0.9610,0.133000,0.0676,0.39200,77.519,313480
7ARrKfixA34QSijMSWgaIK,0.251,0.2340,2,-12.630,0.0301,0.7060,0.000998,0.1050,0.18900,76.959,339750
0puUf8o9DMPD3ew6cX7UZt,0.341,0.3540,2,-15.346,0.0846,0.4780,0.078200,0.1550,0.11700,124.872,206160
5TBWCX4Rgx7eRUs0QMtX7E,0.373,0.4930,1,-9.947,0.0338,0.7840,0.233000,0.1470,0.19500,129.942,184740
...,...,...,...,...,...,...,...,...,...,...,...
7jl9dP7m84gNJP07FfWxAx,0.486,0.6500,9,-30.648,0.5210,0.8890,0.802000,0.3380,0.01360,116.010,228000
4zKJuFzfTOIrLp39Q61ITQ,0.537,0.5530,11,-36.958,0.0661,0.4800,0.161000,0.1990,0.00273,85.026,232000
7wyPXpgEJ72uJVgLXtWvxi,0.197,0.0448,10,-46.219,0.0347,0.7190,0.938000,0.0979,0.22400,82.238,612702


## Calculate % Difference from the top tracks to tracks in top 3 playlists

In [85]:
df1 = df.subtract(y['top_playlist'], axis ='columns')
df1 = df1.divide(y['top_playlist'], axis = 'columns')
df1['%_diff'] = df1.sum(axis=1)
df1['%_diff'] = df1['%_diff'].abs()
df1

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,%_diff
track_id,,,,,,,,,,,,
7qH9Z4dJEN0l9bidizW7fq,-0.318988,-0.460269,0.730769,0.972436,-0.709888,2.411326,25.203871,-0.393525,-0.809409,-0.028766,0.158091,26.755648
28sWYevC75ZrUMJ0tD4zWM,-0.223616,-0.648578,-0.807692,2.133380,-0.699527,3.072403,2.942438,-0.674621,-0.407049,-0.356795,0.409677,4.740018
7ARrKfixA34QSijMSWgaIK,-0.625965,-0.689688,-0.615385,1.170968,-0.716482,1.991797,-0.970417,-0.494604,-0.714113,-0.361441,0.527810,1.497520
0puUf8o9DMPD3ew6cX7UZt,-0.491849,-0.530554,-0.615385,1.637821,-0.203135,1.025607,1.318035,-0.253940,-0.823022,0.036111,-0.072926,1.026765
5TBWCX4Rgx7eRUs0QMtX7E,-0.444163,-0.346223,-0.807692,0.709788,-0.681631,2.322335,5.906677,-0.292446,-0.705037,0.078179,-0.169249,5.570537
...,...,...,...,...,...,...,...,...,...,...,...,...
7jl9dP7m84gNJP07FfWxAx,-0.275773,-0.138022,0.730769,4.268079,3.907409,2.767291,22.773196,0.626893,-0.979428,-0.037420,0.025285,33.668279
4zKJuFzfTOIrLp39Q61ITQ,-0.199773,-0.266656,1.115385,5.352704,-0.377390,1.034083,3.772425,-0.042155,-0.995871,-0.294506,0.043273,9.141517
7wyPXpgEJ72uJVgLXtWvxi,-0.706435,-0.940590,0.923077,6.944576,-0.673153,2.046886,26.804560,-0.528779,-0.661171,-0.317639,1.755238,34.646571


In [86]:
df2 = df1.sort_values('%_diff', ascending = True).head(50)
df2

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,%_diff
track_id,,,,,,,,,,,,
0vCxilr1Z63f1XSQhu2GJN,-0.099931,-0.645926,-0.230769,1.142435,-0.591206,2.669824,-0.986928,-0.364645,-0.602178,-0.067349,-0.181152,0.042174
2CvoR0WNoXehWqrCmdwgQ5,-0.348791,-0.502705,-0.038462,0.758261,-0.381158,1.843478,-0.975604,-0.417592,-0.671759,0.494077,0.142743,0.097511
6lfxq3CG4xtTiEg7opyCyx,-0.603612,-0.920963,-1.000000,2.182540,-0.658083,2.835094,-0.997904,-0.364645,-0.783694,0.507967,-0.091935,0.104765
7rPLZ8Krm6CZIbraFUlnWZ,-0.515692,-0.761299,0.923077,0.830111,-0.707063,2.856282,-0.981236,-0.456098,-0.849191,-0.399136,-0.046485,0.106729
4WzhjxvLP95y7AMDy0Atwb,-0.283224,-0.645926,0.346154,0.589809,-0.680689,2.390138,-1.000000,0.198510,-0.792770,-0.023895,0.023905,0.122012
70K0ezmzYEZeqoSaZMyP7o,-0.429261,-0.429769,0.923077,0.485816,-0.715540,2.462178,-0.999969,-0.581725,-0.736802,-0.112138,0.262526,0.128393
2PwXOevGUSkU8qaYZjgLq2,-0.688552,-0.453639,-1.000000,0.673004,-0.582729,-0.271120,3.446358,-0.594239,-0.754954,-0.284267,0.374903,0.135236
2kJwzbxV2ppxnQoYw4GLBZ,-0.308557,-0.372746,-0.807692,0.733681,0.215078,2.669824,-1.000000,-0.475351,-0.086371,-0.371050,-0.061563,0.135254
1Hck0Y3AgGLIvv9pRuq3ti,-0.350282,-0.558402,0.153846,0.696381,-0.666560,2.059599,-0.999715,-0.099914,-0.907881,-0.008603,0.537028,0.144503


## Creating custom playlist

In [118]:
def create_playlist (sp, username, playlist_name, playlist_description):
    playlists = sp.user_playlist_create(username, playlist_name, description = playlist_description)
    return playlists

In [122]:
today = str(date.today())
playlist_name = 'Custom Playlist ' + today
custom_pl = create_playlist(sp,username, playlist_name, 'Data Based Playlist Created')
code = custom_pl['uri']
link = code.partition("playlist:")[2]

In [123]:
def custom_playlist(sp, username, playlist_id, playlist_tracks):
    results = []
    results += sp.user_playlist_add_tracks(username, playlist_id, tracks = playlist_tracks[0:50])
    

In [124]:
tracklist = df2.index
custom_playlist (sp, username, link , tracklist)